In [1]:
import pandas as pd
import numpy as np
import ast
from scipy import sparse
from sklearn.metrics.pairwise import pairwise_distances, cosine_distances, cosine_similarity

In [2]:
games = pd.concat([pd.read_csv('../data/steam_games_cleaned_1.csv'), pd.read_csv('../data/steam_games_cleaned_2.csv'), pd.read_csv('../data/steam_games_cleaned_3.csv')], axis=0)
print(games.shape)
games.head()

(56763, 506)


,appid,name,genre,developer,publisher,owners,average_forever,median_forever,pos_rating_pct,total_ratings,...,Web Publishing,Well-Written,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports
0,10,Counter-Strike,Action,Valve,Valve,"10,000,000-20,000,000",11666,244,0.974693,189081,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.217837
1,20,Team Fortress Classic,Action,Valve,Valve,"2,000,000-5,000,000",91,18,0.857002,6105,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
2,30,Day of Defeat,Action,Valve,Valve,"5,000,000-10,000,000",403,26,0.900055,5423,...,0.0,0.0,0.0,0.0,0.0,0.015287,0.313376,0.0,0.0,0.000000
3,40,Deathmatch Classic,Action,Valve,Valve,"5,000,000-10,000,000",33,6,0.816689,2193,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000
4,50,Half-Life: Opposing Force,Action,Gearbox Software,Valve,"5,000,000-10,000,000",322,127,0.951749,13119,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000


In [3]:
games['has_tag'] = [games.iloc[i,72:].drop(columns = ['Free to Play','Early Access']).values.max() for i in range(len(games))]
games['has_tag'].value_counts()

1.0    48420
0.0     8343
Name: has_tag, dtype: int64

In [4]:
# recommender will use tag ratios so we'll need to trim out any games missing tags
games = games.loc[games['has_tag'] > 0]

search_df = games.loc[ : , ['appid','name']]
search_df['name'] = [name.lower() for name in search_df['name']]
search_df.set_index('name', inplace=True)

In [5]:
# set appids as our index since we may have similar or duplicate game names
games.set_index('appid', inplace=True)
games.sort_index(inplace=True)
print(games.shape)
games.head()

(48420, 506)


,name,genre,developer,publisher,owners,average_forever,median_forever,pos_rating_pct,total_ratings,price,...,Well-Written,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports,has_tag
appid,,,,,,,,,,,,,,,,,,,,,
10,Counter-Strike,Action,Valve,Valve,"10,000,000-20,000,000",11666,244,0.974693,189081,9.99,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.217837,1.0
20,Team Fortress Classic,Action,Valve,Valve,"2,000,000-5,000,000",91,18,0.857002,6105,4.99,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.0
30,Day of Defeat,Action,Valve,Valve,"5,000,000-10,000,000",403,26,0.900055,5423,4.99,...,0.0,0.0,0.0,0.0,0.015287,0.313376,0.0,0.0,0.000000,1.0
40,Deathmatch Classic,Action,Valve,Valve,"5,000,000-10,000,000",33,6,0.816689,2193,4.99,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.0
50,Half-Life: Opposing Force,Action,Gearbox Software,Valve,"5,000,000-10,000,000",322,127,0.951749,13119,4.99,...,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,1.0


In [6]:
search = 'Beat Hazard'.lower()
titles = search_df[search_df.index.str.contains(search)]
print(titles)

                appid
name                 
beat hazard     49600
beat hazard 2  618740


In [7]:
search_df.loc[search,'appid']

49600

In [8]:
%%time
recommender = games.loc[ : , '1980s' : 'e-sports'].drop(columns = ['Free to Play','Kickstarter','Early Access'])
sparse_rec = sparse.csr_matrix(recommender)
dists = pairwise_distances(sparse_rec, metric='cosine')
recommender_df = pd.DataFrame(dists, columns=recommender.index, index=recommender.index)

Wall time: 5min 2s


In [9]:
top_recommendations = recommender_df[search_df.loc[search,'appid']].sort_values()[1:21]
top_recs_df = pd.DataFrame(top_recommendations)
top_recs_df['name'] = [games.loc[ind]['name'] for ind in top_recs_df.index]
top_recs_df[['name',top_recommendations.name]].rename(columns={'name':'Game Name', top_recommendations.name : f'Similarity to {search} (lower is better)'})

,Game Name,Similarity to beat hazard (lower is better)
appid,,
207750,Symphony,0.099637
618740,Beat Hazard 2,0.140993
1180620,Bullet Beat: Musical Shoot'em up,0.203391
12900,AudioSurf,0.206729
235800,Audiosurf 2,0.227873
67000,The Polynomial - Space of the music,0.231299
290000,DubWars,0.259847
513510,Intralism,0.284387
255370,KickBeat Steam Edition,0.290860


In [10]:
%%time
top100_df = pd.DataFrame(columns = ['games'])
for game_index in list(recommender_df.index):
    top100 = recommender_df[game_index].sort_values()[1:101]
    top100_df.loc[game_index] = str(list(top100.index))
    print(' '*50, end = '\r')
    print(f'{int(round(top100_df.shape[0]/games.shape[0]*100,0))}% complete', end= '\r')
print(' '*50, end = '\r')
top100_df.to_csv('../data/top100_simils.csv')

Wall time: 6min 13s                               


In [11]:
top100_df.loc[search_df.loc[search]]

,games
49600,"[207750, 618740, 1180620, 12900, 235800, 67000..."


In [12]:
top100_df.loc[search_df.loc[search]].values[0][0]

'[207750, 618740, 1180620, 12900, 235800, 67000, 290000, 513510, 255370, 301190, 205080, 357720, 930620, 762500, 222660, 247140, 1020340, 301200, 892930, 823730, 231040, 63700, 1222930, 980360, 921630, 499890, 301540, 268260, 453100, 259530, 329320, 708890, 372690, 1019400, 807960, 865250, 614030, 16300, 878180, 945100, 323040, 1352730, 369030, 1014710, 980610, 463150, 420160, 205060, 246800, 735570, 205070, 381320, 684680, 348280, 397690, 911580, 1291720, 1001970, 926870, 1309980, 1152440, 1048300, 531510, 297110, 977950, 282760, 34920, 351990, 520470, 201570, 818620, 412740, 565660, 394140, 924560, 1132840, 1250350, 446560, 298280, 338000, 342650, 697600, 566780, 438460, 58400, 893030, 1021680, 281860, 727450, 1262180, 691160, 380550, 774171, 920700, 1088960, 266250, 397570, 461230, 218060, 744060]'

In [13]:
results = ast.literal_eval(top100_df.loc[search_df.loc[search]].values[0][0])
search_range = 20
games.loc[results][ : search_range]

,name,genre,developer,publisher,owners,average_forever,median_forever,pos_rating_pct,total_ratings,price,...,Well-Written,Werewolves,Western,Word Game,World War I,World War II,Wrestling,Zombies,e-sports,has_tag
appid,,,,,,,,,,,,,,,,,,,,,
207750,Symphony,"Action, Casual, Indie",Empty Clip Studios,Empty Clip Studios,"100,000-200,000",311,379,0.846369,1432,4.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
618740,Beat Hazard 2,"Action, Casual, Indie",Cold Beam Games,Cold Beam Games,"100,000-200,000",207,231,0.876216,1850,18.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1180620,Bullet Beat: Musical Shoot'em up,"Action, Casual, Indie",TERNOX,TERNOX,"0-20,000",0,0,0.764706,17,4.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
12900,AudioSurf,Indie,Dylan Fitterer,"Codemasters, Electronic Arts","1,000,000-2,000,000",1804,276,0.958470,10354,9.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
235800,Audiosurf 2,Indie,Dylan Fitterer,Dylan Fitterer,"200,000-500,000",973,294,0.817476,5539,14.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
67000,The Polynomial - Space of the music,"Action, Casual, Indie",Dmytry Lavrov,Dmytry Lavrov,"100,000-200,000",15,13,0.735417,480,6.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
290000,DubWars,"Action, Casual, Indie",MURA Interactive,WOBBL3 Entertainment LLC,"50,000-100,000",144,144,0.787234,282,9.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
513510,Intralism,"Action, Casual, Indie",KHB-Soft,KHB-Soft,"500,000-1,000,000",216,265,0.877168,4844,2.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
255370,KickBeat Steam Edition,"Action, Indie",Zen Studios,Zen Studios,"100,000-200,000",119,154,0.664894,752,9.99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
search_df.to_csv('../data/search_keys.csv')

In [15]:
search_df.head()

,appid
name,
counter-strike,10
team fortress classic,20
day of defeat,30
deathmatch classic,40
half-life: opposing force,50


In [16]:
search = 'Counter'.lower()
titles = list(search_df[search_df.index.str.contains(search)].index)
list(games.loc[search_df.loc[titles]['appid'].values]['name'].values)

['Counter-Strike',
 'Counter-Strike: Condition Zero',
 'Counter-Strike: Condition Zero',
 'Counter-Strike: Condition Zero',
 'Counter-Strike: Condition Zero',
 'Counter-Strike: Source',
 'Counter-Strike: Global Offensive',
 'Harvest: Massive Encounter',
 'Serious Sam HD: The First Encounter',
 'Serious Sam HD: The Second Encounter',
 'Serious Sam Classic: The First Encounter',
 'Serious Sam Classic: The Second Encounter',
 'Serious Sam: The Random Encounter',
 'Counter-Strike Nexon: Studio',
 'Counter Spell',
 "Ghost Encounters: Deadwood - Collector's Edition",
 'Jet Racing Extreme: The First Encounter',
 'CTU: Counter Terrorism Unit',
 'OASE - Other Age Second Encounter',
 'Rencounter',
 'Pub Encounter',
 'CounterAttack',
 'Counter Agents',
 'Counter Fight',
 'Serious Sam VR: The First Encounter',
 'Serious Sam VR: The Second Encounter',
 'Counter Fight: Samurai Edition',
 'Encounter of Galaxies',
 'Elite Encounter',
 'Graze Counter',
 'Last Encounter',
 'Hallowed Encounter',
 'The Ba

In [24]:
col_count = games.loc[:, '1980s' : 'e-sports'].columns
for col in range(len(col_count)):
    print(col, col_count[col])

0 1980s
1 1990's
2 2.5D
3 2D
4 2D Fighter
5 2D Platformer
6 360 Video
7 3D
8 3D Fighter
9 3D Platformer
10 3D Vision
11 4 Player Local
12 4X
13 6DOF
14 8-bit Music
15 ATV
16 Abstract
17 Action
18 Action RPG
19 Action RTS
20 Action Roguelike
21 Action-Adventure
22 Addictive
23 Adventure
24 Agriculture
25 Aliens
26 Alternate History
27 Ambient
28 America
29 Animation & Modeling
30 Anime
31 Arcade
32 Archery
33 Arena Shooter
34 Artificial Intelligence
35 Assassin
36 Asymmetric VR
37 Asynchronous Multiplayer
38 Atmospheric
39 Audio Production
40 Auto Battler
41 Automation
42 Automobile Sim
43 BMX
44 Base-Building
45 Baseball
46 Based On A Novel
47 Basketball
48 Batman
49 Battle Royale
50 Beat 'em up
51 Beautiful
52 Benchmark
53 Bikes
54 Blood
55 Board Game
56 Boss Rush
57 Bowling
58 Boxing
59 Building
60 Bullet Hell
61 Bullet Time
62 CRPG
63 Capitalism
64 Card Battler
65 Card Game
66 Cartoon
67 Cartoony
68 Casual
69 Cats
70 Character Action Game
71 Character Customization
72 Chess
73 Choic